In [1]:
#import important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
# from sklearn.tree import DecisionTreeClassifier  
from sklearn.metrics import accuracy_score  
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df=pd.read_csv(r"C:\Users\arwah\OneDrive\Desktop\HealthCare Project\datasets\Titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [6]:
df.fillna()

ValueError: Must specify a fill 'value' or 'method'.

In [4]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,0.363636,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.481622,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,0.000000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,0.000000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,1.000000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,1.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [5]:
numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
print("Numeric Columns:", numeric_columns)
print("Categorical Columns:", categorical_columns)

Numeric Columns: ['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
Categorical Columns: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


In [316]:
outliers_summary = {}
for col in numeric_columns:
    Q1 = df[col].quantile(0.25)  
    Q3 = df[col].quantile(0.75)  
    IQR = Q3 - Q1                  
   
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outliers_count = outliers.shape[0]
    outliers_summary[col] = outliers_count

for col, count in outliers_summary.items():
    print(f"Column: {col}, Outliers: {count}")

Column: PatientID, Outliers: 0
Column: HeightInMeters, Outliers: 0
Column: WeightInKilograms, Outliers: 638
Column: BMI, Outliers: 968
Column: HadHeartAttack, Outliers: 13201
Column: HadAngina, Outliers: 14617
Column: HadStroke, Outliers: 9928
Column: HadAsthma, Outliers: 35292
Column: HadSkinCancer, Outliers: 20252
Column: HadCOPD, Outliers: 18602
Column: HadDepressiveDisorder, Outliers: 48896
Column: HadKidneyDisease, Outliers: 11029
Column: HadArthritis, Outliers: 0
Column: DeafOrHardOfHearing, Outliers: 20540
Column: BlindOrVisionDifficulty, Outliers: 11987
Column: DifficultyConcentrating, Outliers: 25501
Column: DifficultyWalking, Outliers: 35391
Column: DifficultyDressingBathing, Outliers: 8204
Column: DifficultyErrands, Outliers: 16056
Column: ChestScan, Outliers: 0
Column: AlcoholDrinkers, Outliers: 0
Column: HIVTesting, Outliers: 0
Column: FluVaxLast12, Outliers: 0
Column: PneumoVaxEver, Outliers: 0
Column: HighRiskLastYear, Outliers: 10176
Column: CovidPos, Outliers: 0


In [315]:
import numpy as np

for column in df.select_dtypes(include=[np.number]).columns:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = (df[column] < lower_bound) | (df[column] > upper_bound)
    mean_value = df[column].mean()
    df[column] = np.where(outliers, mean_value, df[column])

for column in df.select_dtypes(include=['object']).columns:

    mode_value = df[column].mode()[0]
    
 
    outliers = df[column].isnull()  
    df[column] = np.where(outliers, mode_value, df[column])


df.head()


,PatientID,State,Sex,GeneralHealth,AgeCategory,HeightInMeters,WeightInKilograms,BMI,HadHeartAttack,HadAngina,...,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,1.0,Alabama,Female,Fair,Age 75 to 79,1.63,84.820000,32.099998,0.0,0.003784,...,Never used e-cigarettes in my entire life,1.0,"White only, Non-Hispanic",0.0,0.0,0.0,1.0,"No, did not receive any tetanus shot in the pa...",0.0,1.0
1,2.0,Alabama,Female,Very good,Age 65 to 69,1.60,71.669998,27.990000,0.0,0.000000,...,Never used e-cigarettes in my entire life,0.0,"White only, Non-Hispanic",0.0,0.0,1.0,1.0,"Yes, received Tdap",0.0,0.0
2,3.0,Alabama,Male,Excellent,Age 60 to 64,1.78,71.209999,22.530001,0.0,0.000000,...,Never used e-cigarettes in my entire life,0.0,"White only, Non-Hispanic",1.0,0.0,0.0,0.0,"Yes, received tetanus shot but not sure what type",0.0,0.0
3,4.0,Alabama,Male,Very good,Age 70 to 74,1.78,95.250000,30.129999,0.0,0.000000,...,Never used e-cigarettes in my entire life,0.0,"White only, Non-Hispanic",0.0,0.0,1.0,1.0,"Yes, received tetanus shot but not sure what type",0.0,0.0
4,5.0,Alabama,Female,Good,Age 50 to 54,1.68,78.019997,27.760000,0.0,0.000000,...,Never used e-cigarettes in my entire life,1.0,"Black only, Non-Hispanic",0.0,0.0,1.0,0.0,"No, did not receive any tetanus shot in the pa...",0.0,0.0


In [317]:
df.value_counts()

PatientID  State        Sex     GeneralHealth  AgeCategory      HeightInMeters  WeightInKilograms  BMI        HadHeartAttack  HadAngina  HadStroke  HadAsthma  HadSkinCancer  HadCOPD  HadDepressiveDisorder  HadKidneyDisease  HadArthritis  HadDiabetes                              DeafOrHardOfHearing  BlindOrVisionDifficulty  DifficultyConcentrating  DifficultyWalking  DifficultyDressingBathing  DifficultyErrands  SmokerStatus                           ECigaretteUsage                            ChestScan  RaceEthnicityCategory     AlcoholDrinkers  HIVTesting  FluVaxLast12  PneumoVaxEver  TetanusLast10Tdap                                          HighRiskLastYear  CovidPos
1.0        Alabama      Female  Fair           Age 75 to 79     1.63            84.820000          32.099998  0.0             0.003784   0.0        0.022057   0.007263       0.0      0.000000               0.0               1.0           Yes                                      0.0                  0.000000              

In [318]:
df.isna().sum()

PatientID                    0
State                        0
Sex                          0
GeneralHealth                0
AgeCategory                  0
HeightInMeters               0
WeightInKilograms            0
BMI                          0
HadHeartAttack               0
HadAngina                    0
HadStroke                    0
HadAsthma                    0
HadSkinCancer                0
HadCOPD                      0
HadDepressiveDisorder        0
HadKidneyDisease             0
HadArthritis                 0
HadDiabetes                  0
DeafOrHardOfHearing          0
BlindOrVisionDifficulty      0
DifficultyConcentrating      0
DifficultyWalking            0
DifficultyDressingBathing    0
DifficultyErrands            0
SmokerStatus                 0
ECigaretteUsage              0
ChestScan                    0
RaceEthnicityCategory        0
AlcoholDrinkers              0
HIVTesting                   0
FluVaxLast12                 0
PneumoVaxEver                0
TetanusL

In [153]:
# mode_value = df['Alcohol Consumption'].mode()[0]  # القيمة الأكثر تكرارًا
# df['Alcohol Consumption'] = df['Alcohol Consumption'].fillna(mode_value)


In [227]:
#check null values
df.isna()
rows_with_nan = df[df.isna().any(axis=1)]
print(rows_with_nan)

print(df.isnull().sum())
missing_percentage = df.isnull().mean() * 100
print(missing_percentage)

Empty DataFrame
Columns: [General_Health, Checkup, Exercise, Heart_Disease, Skin_Cancer, Other_Cancer, Depression, Diabetes, Arthritis, Sex, Age_Category, Height_(cm), Weight_(kg), BMI, Smoking_History, Alcohol_Consumption, Fruit_Consumption, Green_Vegetables_Consumption, FriedPotato_Consumption]
Index: []
General_Health                  0
Checkup                         0
Exercise                        0
Heart_Disease                   0
Skin_Cancer                     0
Other_Cancer                    0
Depression                      0
Diabetes                        0
Arthritis                       0
Sex                             0
Age_Category                    0
Height_(cm)                     0
Weight_(kg)                     0
BMI                             0
Smoking_History                 0
Alcohol_Consumption             0
Fruit_Consumption               0
Green_Vegetables_Consumption    0
FriedPotato_Consumption         0
dtype: int64
General_Health                  0

In [ ]:
# df.fillna("Occasional")

,Age,Gender,Heart rate,Systolic blood pressure,Diastolic blood pressure,Blood sugar,CK-MB,Troponin,Result,Cholesterol,BMI,Smoking,Exercise Frequency,Diet Type,Alcohol Consumption,Family History,Stress Level,Occupation Type
0,1.353606,-1.388308,-0.415639,-0.094117,-0.027127,-0.778559,-0.221563,-0.296767,0.513093,0.508563,-0.471250,-0.637705,1.493218,0.133796,-0.733319,1.536580,-0.257845,-1.277599
1,0.263489,0.720301,-0.150652,-0.588952,-0.310443,-1.089787,-0.198935,-0.229230,0.513093,-0.132727,-0.311983,1.568123,-1.498204,1.564767,0.757167,0.109858,-0.257845,0.155651
3,-0.826629,0.720301,-0.283146,-0.398631,-0.027127,-0.886812,-0.258309,-0.311486,-1.948965,1.643152,-0.013357,1.568123,0.994648,1.564767,-0.733319,0.109858,-1.594522,0.155651
5,-1.989421,0.720301,-0.349393,0.591038,0.185359,-0.697369,-0.215335,-0.308023,0.513093,-0.231387,-0.789785,-0.637705,-0.999633,0.133796,0.757167,0.109858,1.078832,0.155651
9,0.990234,0.720301,-0.371475,-0.322502,0.185359,-0.670306,-0.158452,-0.296767,0.513093,0.533228,0.802889,-0.637705,0.994648,-1.297176,-0.733319,0.109858,-1.594522,0.155651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,-1.190001,0.720301,0.268911,-0.208310,-0.168785,0.520478,-0.271180,-0.308023,-1.948965,0.187918,-0.471250,-0.637705,-0.002493,-1.297176,0.757167,0.109858,1.078832,0.155651
9992,-1.335350,-1.388308,0.268911,0.286524,0.185359,-0.521458,-0.319925,-0.270791,0.513093,0.656553,0.862614,-0.637705,0.994648,-1.297176,-2.223806,1.536580,1.078832,-1.277599
9993,-1.190001,0.720301,0.290994,-0.284438,0.539504,-0.873281,-0.261630,-0.319279,0.513093,-0.946672,1.001973,-0.637705,-0.002493,0.133796,-0.733319,0.109858,1.078832,0.155651
9997,-0.463256,0.720301,-0.062323,0.552973,0.185359,-0.332015,-0.199142,-0.307157,-1.948965,-0.700022,0.364903,1.568123,-0.002493,0.133796,0.757167,-1.316865,1.078832,0.155651


In [239]:
df.isna().sum()


General_Health                  0
Checkup                         0
Exercise                        0
Heart_Disease                   0
Skin_Cancer                     0
Other_Cancer                    0
Depression                      0
Diabetes                        0
Arthritis                       0
Sex                             0
Age_Category                    0
Height_(cm)                     0
Weight_(kg)                     0
BMI                             0
Smoking_History                 0
Alcohol_Consumption             0
Fruit_Consumption               0
Green_Vegetables_Consumption    0
FriedPotato_Consumption         0
dtype: int64

In [319]:
enc=LabelEncoder()
for col in categorical_columns:
    df[col] = enc.fit_transform(df[col])

df    

,PatientID,State,Sex,GeneralHealth,AgeCategory,HeightInMeters,WeightInKilograms,BMI,HadHeartAttack,HadAngina,...,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,1.0,0,0,1,11,1.63,84.820000,32.099998,0.000000,0.003784,...,0,1.0,4,0.0,0.0,0.0,1.0,0,0.000000,1.0
1,2.0,0,0,4,9,1.60,71.669998,27.990000,0.000000,0.000000,...,0,0.0,4,0.0,0.0,1.0,1.0,1,0.000000,0.0
2,3.0,0,1,0,8,1.78,71.209999,22.530001,0.000000,0.000000,...,0,0.0,4,1.0,0.0,0.0,0.0,2,0.000000,0.0
3,4.0,0,1,4,10,1.78,95.250000,30.129999,0.000000,0.000000,...,0,0.0,4,0.0,0.0,1.0,1.0,2,0.000000,0.0
4,5.0,0,0,2,6,1.68,78.019997,27.760000,0.000000,0.000000,...,0,1.0,0,0.0,0.0,1.0,0.0,0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237625,237626.0,40,0,2,8,1.57,90.720001,36.580002,0.000000,0.000000,...,1,0.0,1,0.0,0.0,1.0,0.0,0,0.000000,0.0
237626,237627.0,40,0,2,7,1.70,72.570000,25.059999,0.000000,0.003784,...,1,0.0,1,0.0,1.0,0.0,0.0,1,0.000000,0.0
237627,237628.0,40,1,1,5,1.75,70.309998,22.889999,0.003086,0.003784,...,1,1.0,1,0.0,1.0,1.0,1.0,1,0.001834,0.0
237628,237629.0,40,0,4,1,1.57,46.720001,18.840000,0.000000,0.000000,...,0,0.0,1,0.0,1.0,0.0,0.0,0,0.000000,1.0


In [320]:
#data scalling
numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
print(df[numeric_columns].head())
df


   PatientID     State       Sex  GeneralHealth  AgeCategory  HeightInMeters  \
0  -1.732044 -1.781416 -0.962995      -0.870032     1.228950       -0.717258   
1  -1.732029 -1.781416 -0.962995       1.169102     0.653206       -1.004647   
2  -1.732014 -1.781416  1.038427      -1.549743     0.365334        0.719686   
3  -1.732000 -1.781416  1.038427       1.169102     0.941078        0.719686   
4  -1.731985 -1.781416 -0.962995      -0.190321    -0.210409       -0.238277   

   WeightInKilograms       BMI  HadHeartAttack  HadAngina  ...  \
0           0.174247  0.837216       -0.242529   3.906034  ...   
1          -0.560506  0.022374       -0.242529  -0.256014  ...   
2          -0.586208 -1.060117       -0.242529  -0.256014  ...   
3           0.757021  0.446647       -0.242529  -0.256014  ...   
4          -0.205701 -0.023225       -0.242529  -0.256014  ...   

   ECigaretteUsage  ChestScan  RaceEthnicityCategory  AlcoholDrinkers  \
0        -0.470762   1.158552               0.520

,PatientID,State,Sex,GeneralHealth,AgeCategory,HeightInMeters,WeightInKilograms,BMI,HadHeartAttack,HadAngina,...,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,-1.732044,-1.781416,-0.962995,-0.870032,1.228950,-0.717258,0.174247,0.837216,-0.242529,3.906034,...,-0.470762,1.158552,0.520552,-1.095070,-0.722058,-1.065986,1.206752,-1.163011,-0.211515,1.542425
1,-1.732029,-1.781416,-0.962995,1.169102,0.653206,-1.004647,-0.560506,0.022374,-0.242529,-0.256014,...,-0.470762,-0.863146,0.520552,-1.095070,-0.722058,0.938099,1.206752,-0.133353,-0.211515,-0.648330
2,-1.732014,-1.781416,1.038427,-1.549743,0.365334,0.719686,-0.586208,-1.060117,-0.242529,-0.256014,...,-0.470762,-0.863146,0.520552,0.913184,-0.722058,-1.065986,-0.828671,0.896304,-0.211515,-0.648330
3,-1.732000,-1.781416,1.038427,1.169102,0.941078,0.719686,0.757021,0.446647,-0.242529,-0.256014,...,-0.470762,-0.863146,0.520552,-1.095070,-0.722058,0.938099,1.206752,0.896304,-0.211515,-0.648330
4,-1.731985,-1.781416,-0.962995,-0.190321,-0.210409,-0.238277,-0.205701,-0.023225,-0.242529,-0.256014,...,-0.470762,1.158552,-2.480953,-1.095070,-0.722058,0.938099,-0.828671,-1.163011,-0.211515,-0.648330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237625,1.731985,0.806987,-0.962995,-0.190321,0.365334,-1.292035,0.503909,1.725415,-0.242529,-0.256014,...,1.064335,-0.863146,-1.730577,-1.095070,-0.722058,0.938099,-0.828671,-1.163011,-0.211515,-0.648330
237626,1.732000,0.806987,-0.962995,-0.190321,0.077462,-0.046684,-0.510219,-0.558523,-0.242529,3.906034,...,1.064335,-0.863146,-1.730577,-1.095070,1.384931,-1.065986,-0.828671,-0.133353,-0.211515,-0.648330
237627,1.732014,0.806987,1.038427,-0.870032,-0.498281,0.432297,-0.636496,-0.988744,4.123216,3.906034,...,1.064335,1.158552,-1.730577,-1.095070,1.384931,0.938099,1.206752,-0.133353,4.727791,-0.648330
237628,1.732029,0.806987,-0.962995,1.169102,-1.649769,-1.292035,-1.954582,-1.791691,-0.242529,-0.256014,...,-0.470762,-0.863146,-1.730577,-1.095070,1.384931,-1.065986,-0.828671,-1.163011,-0.211515,1.542425


In [321]:
outliers_summary = {}
for col in numeric_columns:
    Q1 = df[col].quantile(0.25)  
    Q3 = df[col].quantile(0.75)  
    IQR = Q3 - Q1                  
   
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outliers_count = outliers.shape[0]
    outliers_summary[col] = outliers_count

for col, count in outliers_summary.items():
    print(f"Column: {col}, Outliers: {count}")

Column: PatientID, Outliers: 0
Column: State, Outliers: 0
Column: Sex, Outliers: 0
Column: GeneralHealth, Outliers: 0
Column: AgeCategory, Outliers: 0
Column: HeightInMeters, Outliers: 0
Column: WeightInKilograms, Outliers: 638
Column: BMI, Outliers: 968
Column: HadHeartAttack, Outliers: 13201
Column: HadAngina, Outliers: 14617
Column: HadStroke, Outliers: 9928
Column: HadAsthma, Outliers: 35292
Column: HadSkinCancer, Outliers: 20252
Column: HadCOPD, Outliers: 18602
Column: HadDepressiveDisorder, Outliers: 48896
Column: HadKidneyDisease, Outliers: 11029
Column: HadArthritis, Outliers: 0
Column: HadDiabetes, Outliers: 40167
Column: DeafOrHardOfHearing, Outliers: 20540
Column: BlindOrVisionDifficulty, Outliers: 11987
Column: DifficultyConcentrating, Outliers: 25501
Column: DifficultyWalking, Outliers: 35391
Column: DifficultyDressingBathing, Outliers: 8204
Column: DifficultyErrands, Outliers: 16056
Column: SmokerStatus, Outliers: 21148
Column: ECigaretteUsage, Outliers: 54184
Column: Che

In [322]:
import pandas as pd

# تطبيق IQR على جميع الأعمدة العددية
def remove_outliers(df):
    numeric_cols = df.select_dtypes(include=['number']).columns  # اختيار الأعمدة العددية
    
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    return df

df = remove_outliers(df)


In [323]:
outliers_summary = {}
for col in numeric_columns:
    Q1 = df[col].quantile(0.25)  
    Q3 = df[col].quantile(0.75)  
    IQR = Q3 - Q1                  
   
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outliers_count = outliers.shape[0]
    outliers_summary[col] = outliers_count

for col, count in outliers_summary.items():
    print(f"Column: {col}, Outliers: {count}")

Column: PatientID, Outliers: 0
Column: State, Outliers: 0
Column: Sex, Outliers: 0
Column: GeneralHealth, Outliers: 0
Column: AgeCategory, Outliers: 0
Column: HeightInMeters, Outliers: 3
Column: WeightInKilograms, Outliers: 160
Column: BMI, Outliers: 511
Column: HadHeartAttack, Outliers: 0
Column: HadAngina, Outliers: 0
Column: HadStroke, Outliers: 0
Column: HadAsthma, Outliers: 0
Column: HadSkinCancer, Outliers: 0
Column: HadCOPD, Outliers: 0
Column: HadDepressiveDisorder, Outliers: 0
Column: HadKidneyDisease, Outliers: 0
Column: HadArthritis, Outliers: 0
Column: HadDiabetes, Outliers: 0
Column: DeafOrHardOfHearing, Outliers: 0
Column: BlindOrVisionDifficulty, Outliers: 0
Column: DifficultyConcentrating, Outliers: 0
Column: DifficultyWalking, Outliers: 0
Column: DifficultyDressingBathing, Outliers: 0
Column: DifficultyErrands, Outliers: 0
Column: SmokerStatus, Outliers: 4965
Column: ECigaretteUsage, Outliers: 0
Column: ChestScan, Outliers: 0
Column: RaceEthnicityCategory, Outliers: 29

In [324]:
X = df.drop('HadHeartAttack', axis=1) 
y = df['HadHeartAttack']  

In [325]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


In [326]:
from sklearn.linear_model import LogisticRegression
logR=LogisticRegression()
logR.fit(X_train,y_train)
logR_predict=logR.predict(X_test)
Tr_score=logR.score(X_train,y_train)
Tst_score=logR.score(X_test,y_test)
print("Complete LogisticRegression Training !",Tr_score)
print("Complete LogisticRegression Test!",Tst_score)

ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [327]:
from sklearn.model_selection import train_test_split as tts
Xtrain,Xtest,Ytrain,Ytest=tts(X,y,test_size=0.30,random_state=1)

In [328]:
from sklearn.svm import SVC

# Choose regression method and set hyperparameter
svc=SVC(kernel='rbf',)
svc.fit(Xtrain,Ytrain)
svc_predict=svc.predict(Xtest)
svc_train_score=svc.score(Xtrain,Ytrain)
svc_test_score=svc.score(Xtest,Ytest)
print("Complete SVC Training !",svc_train_score)
print("Complete SVC Test!",svc_test_score)

ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [329]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
test_pred = model.predict(X_test)

print("R² Score:", r2_score(y_test, test_pred))


R² Score: 1.0


In [330]:
from sklearn.model_selection import train_test_split


# تدريب النموذج على البيانات التدريبية
model.fit(X_train, y_train)

# اختبار النموذج على البيانات الاختبارية
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy}")


Accuracy: 1.0


In [331]:
from sklearn.model_selection import cross_val_score

# تطبيق Cross-Validation على النموذج
scores = cross_val_score(model, df.drop('HeartDisease', axis=1), df['HeartDisease'], cv=5)  # استخدام 5-fold cross-validation
print(f"Cross-Validation Scores: {scores}")
print(f"Mean Cross-Validation Score: {scores.mean()}")

KeyError: "['HeartDisease'] not found in axis"

In [332]:
from sklearn.ensemble import GradientBoostingRegressor

# تجربة معلمات مختلفة
gb_model = GradientBoostingRegressor(n_estimators=200, max_depth=3, learning_rate=0.1, random_state=42)

# تدريب النموذج
gb_model.fit(X_train, y_train)

# التنبؤ بالقيم
y_pred_gb = gb_model.predict(X_test)

# حساب الـ R² و MSE
print("Tuned Gradient Boosting R²:", r2_score(y_test, y_pred_gb))
print("Tuned Gradient Boosting MSE:", mean_squared_error(y_test, y_pred_gb))

Tuned Gradient Boosting R²: 1.0
Tuned Gradient Boosting MSE: 0.0


In [333]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Mean Squared Error:", mean_squared_error(y_test, y_pred_rf))
print("Random Forest R2",r2_score(y_test, y_pred_rf))



Random Forest Mean Squared Error: 7.333555345384269e-28
Random Forest R2 -951948.9095432508


In [334]:

from sklearn.metrics import r2_score

# حساب R^2 (معامل التحديد)
r2 = r2_score(y_test, y_pred_rf)
print("R^2:", r2)


R^2: -951948.9095432508
